In [1]:
%load_ext autoreload
%autoreload 2

# import library

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from package.utils import DotDict, timer, get_config, load_data, merge_data

from package.preprocessing import CONFIG_PATH

conf = get_config(CONFIG_PATH)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


function: get_config is starting...
function: get_config successfully executed at 0.0048291683197021484s


# import dataset

In [4]:
for col in conf.PREMIUM_INPUT:
    print(col)

total_premium_saving
total_premium_wholelife
total_premium_health
total_premium_ci
total_premium


In [5]:
%%time
cols = ['party_rk', 'policy_no', 'plan_cd', 'plan_index', 'issue_dt', 'effective_dt', 'policy_status1', 'policy_status2']
path = 's3://tli-crm-segmentation/data-zones/neutral/raw/ds_mst_policy_profile_202505091742.csv'
dtypes = {'policy_no':str, 'party_rk':str}
policy_df = pd.read_csv(path, usecols=cols, dtype=dtypes)
# policy_df = load_data(path=path, dtype=dtypes)

<timed exec>:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 1min 11s, sys: 16.8 s, total: 1min 27s
Wall time: 5min 1s


In [ ]:
path = 's3://tli-crm-segmentation/data-zones/neutral/raw/tay_ds_mst_rider_profile_202504090908.csv'
dtypes = {'policy_no':str}
rider_df = load_data(path=path, dtype=dtypes)

# feature

In [ ]:
from package.preprocessing.baseplan import filter_active_policy, prep_feature_policy
from package.preprocessing.rider import filter_active_rider, prep_feature_rider
from package.preprocessing.beneficiary import prep_feature_beneficiary

## base plan

In [6]:
from package.preprocessing.baseplan import filter_active_policy, prep_feature_policy

policy_df = filter_active_policy(data=policy_df, conf=conf)

In [ ]:
from package.preprocessing.get_anp import get_premium_baseplan, get_premium_rider, get_anp
from package.preprocessing.driver import impute_0anp_driver

proxy = get_premium_baseplan(data=policy_df)
proxy = get_anp(data=proxy)
proxy = impute_0anp_driver(data=proxy)

In [ ]:
proxy.head()

In [ ]:
from package.preprocessing.baseplan import filter_active_policy, prep_feature_policy

feature_df = prep_feature_policy(data=proxy, conf=conf)

In [ ]:
feature_df.head()

### beneficiary

In [ ]:
from package.preprocessing.beneficiary import prep_feature_beneficiary

proxy = prep_feature_beneficiary(data=proxy, conf=conf)

In [ ]:
proxy.head()

In [ ]:
feature_df = pd.merge(left=feature_df, right=proxy, how='left', left_index=True, right_index=True)

In [ ]:
feature_df.head()

In [80]:
from package.preprocessing.driver import split_onlytermpa_driver

split_onlytermpa_driver(data=feature_df)

(            policy_count_saving  total_premium_saving  policy_count_wholelife  \
 party_rk                                                                        
 8000000052                  1.0                   0.0                     NaN   
 8000000072                  1.0                   0.0                     NaN   
 8000000083                  1.0                7000.0                     NaN   
 8000000085                  1.0                   0.0                     NaN   
 8000000096                  1.0                   0.0                     NaN   
 ...                         ...                   ...                     ...   
 8011617872                  NaN                   NaN                     1.0   
 8011617873                  NaN                   NaN                     1.0   
 8011617875                  NaN                   NaN                     1.0   
 8011617876                  NaN                   NaN                     1.0   
 8011617879     

## rider

In [6]:
from package.preprocessing.baseplan import filter_active_policy, prep_feature_policy

policy_df = filter_active_policy(data=policy_df, conf=conf)

In [6]:
from package.preprocessing.rider import filter_active_rider, prep_feature_rider

rider_df = filter_active_rider(data=rider_df, conf=conf)

In [56]:
%%time
from package.preprocessing.rider import get_partyrk_rider

proxy = get_partyrk_rider(policy_df=policy_df, rider_df=rider_df)

CPU times: user 42.9 s, sys: 3.44 s, total: 46.3 s
Wall time: 46.1 s


In [57]:
from package.preprocessing.get_anp import get_premium_baseplan, get_premium_rider, get_anp

proxy = get_premium_rider(data=proxy)
proxy = get_anp(data=proxy)

function: get_premium_rider is starting...
function: get_premium_rider successfully executed at 2.8587546348571777s
function: get_anp is starting...
function: get_anp successfully executed at 1.2988500595092773s


In [58]:
from package.preprocessing.rider import filter_active_rider, prep_feature_rider

proxy = prep_feature_rider(data=proxy, conf=conf)

function: prep_feature_rider is starting...
function: prep_feature_rider successfully executed at 19.45373249053955s


In [60]:
proxy.head()

,policy_count_health,total_premium_health,policy_count_ci,total_premium_ci,policy_count_accident_medical,total_premium_accident_medical,policy_count_accident_death,total_premium_accident_death
party_rk,,,,,,,,
8000000052,3.0,19677.0,NaN,NaN,NaN,NaN,NaN,NaN
8000000085,3.0,12867.0,NaN,NaN,NaN,NaN,2.0,1800.0
8000000089,2.0,4200.0,NaN,NaN,NaN,NaN,NaN,NaN
8000000105,4.0,19141.0,NaN,NaN,NaN,NaN,NaN,NaN
8000000106,4.0,33455.0,NaN,NaN,NaN,NaN,1.0,3000.0


- กรอง data rider n,s
- join policy แบบยังไม่ต้องกรอง policy status

- ดู policy ที่ inactive แล้วมี rider จาก is_rider 1 มาดูว่า rider status เป็นแบบไหน
- อาจมี rider ที่ status ผิดอยู่เพราะเอา active rider ไป join แล้วมีไม่หมด
- ถ้า baseplan ไม่ active จะไม่มี rider

# group customers

In [27]:
path = 's3://tli-crm-segmentation/data-zones/neutral/processed/feature.csv'
feature_df = pd.read_csv(path)
feature_df

,party_rk,policy_count_saving,total_premium_saving,policy_count_wholelife,total_premium_wholelife,policy_count_other,total_premium_other,is_bene_parent,is_bene_child,is_bene_spouse,policy_count_health,total_premium_health,policy_count_ci,total_premium_ci,policy_count_accident,total_premium_accident,total_premium
0,8000000052,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.5,3.0,19677.0,0.0,0.0,0.0,0.0,19677.0
1,8000000072,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,8000000083,1.0,7000.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7000.0
3,8000000085,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,12867.0,0.0,0.0,3.0,2025.0,14892.0
4,8000000096,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347425,8011596763,0.0,0.0,0.0,0.0,1.0,1116.0,0.0,0.0,0.0,1.0,2160.0,2.0,1464.0,0.0,0.0,4740.0
3347426,8011596769,0.0,0.0,0.0,0.0,1.0,4452.0,0.0,0.0,0.0,1.0,1656.0,1.0,10428.0,0.0,0.0,16536.0
3347427,8011596789,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,5400.0,5400.0
3347428,8011596790,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,9180.0,9180.0


In [14]:
from package.grouping import CONFIG_PATH

conf = get_config(config_path=CONFIG_PATH)

feature_df = feature_df.set_index('party_rk')

function: get_config is starting...
function: get_config successfully executed at 0.0007076263427734375s


In [78]:
proxy = feature_df.copy()

In [17]:
from package.grouping.group_customer import group_customer

group99pct_df, group0anp_df, group1hold_df, other_df = group_customer(data=feature_df)

function: group_customer is starting...
function: group_customer successfully executed at 2.3786911964416504s


# debug

หาว่า rider ที่ไม่ได้ใช้ เช่น industrial อยู่ใน policy อะไร

In [9]:
group_list = ['Term Rider', 'Industrial', 'Ordinary']

In [11]:
proxy = rider_df[rider_df['rider_group'].isin(group_list)].copy()

In [22]:
for group in group_list:
    mask = proxy['rider_group']==group
    x = proxy[mask].copy()
    for category in x['rider_category'].unique():
        print(f'{group}, {category}')
        display(x.loc[x['rider_category']==category, 'rider_cd'].unique())

Term Rider, Term Rider


array(['ฉพ1', 'ฉพ', 'ฉพ2', 'ฉพ3', 'ฉพ4'], dtype=object)

Term Rider, Family Income for Death


array(['701', 'I01', 'I03'], dtype=object)

Industrial, Modify Endownment


array(['ME', 'MC'], dtype=object)

Ordinary, Modify Endownment


array(['SK', 'DH'], dtype=object)

Ordinary, Whole Life


array(['AD'], dtype=object)

In [26]:
proxy[proxy['rider_cd']=='ME']['rider_group'].unique()

array(['Industrial'], dtype=object)

In [30]:
mask = rider_df['rider_group'].isin(['Accident Rider'])
mask |= rider_df['rider_cd']=='ME'
x = rider_df[mask]

In [32]:
x.head()

,policy_no,rider_cd,rider_group,rider_category,sa,premium,extrapremium,rider_status
4,21080165,อ02,Accident Rider,Accident Death & Dismemberment,100000.0,700.0,0,N
5,21080168,อ02,Accident Rider,Accident Death & Dismemberment,200000.0,624.0,0,N
10,21080192,อ02,Accident Rider,Accident Death & Dismemberment,100000.0,189.0,0,N
15,21080220,อ02,Accident Rider,Accident Death & Dismemberment,100000.0,600.0,0,N
16,21080222,อ02,Accident Rider,Accident Death & Dismemberment,100000.0,600.0,0,N


In [34]:
x['rider_group'].unique()

array(['Accident Rider', 'Industrial', 'Personal Accident'], dtype=object)